In [1]:
import sys
import langchain
import langchain_openai

print(f"Python: {sys.version}")
print(f"LangChain: {langchain.__version__}")

Python: 3.13.3 (tags/v3.13.3:6280bb5, Apr  8 2025, 14:47:33) [MSC v.1943 64 bit (AMD64)]
LangChain: 0.3.27


In [2]:
import os
import warnings
import httpx
from dotenv import load_dotenv

warnings.filterwarnings("ignore", message="Unverified HTTPS request")
load_dotenv(override=True)

# Cliente HTTP sem verificação SSL
GLOBAL_HTTP_CLIENT = httpx.Client(verify=False, timeout=60)

# Verifica API Key
api_key = os.getenv("OPENAI_API_KEY")
print(f"API Key carregada: {api_key[:7]}...{api_key[-4:]}" if api_key else "❌ API Key não encontrada")

API Key carregada: sk-proj...40wA


In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.7,
    http_client=GLOBAL_HTTP_CLIENT
)

resposta = llm.invoke("Responda apenas: Conexão estabelecida!")
print(resposta.content)

Conexão estabelecida!


In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7, http_client=GLOBAL_HTTP_CLIENT)

resposta = llm.invoke("Explique em uma frase o que é LangChain.")
print(resposta.content)



LangChain é uma plataforma descentralizada que visa conectar tradutores e clientes de tradução de forma direta e segura, utilizando contratos inteligentes e tecnologia blockchain.


In [5]:
# Testando diferentes temperaturas
from langchain_openai import ChatOpenAI

pergunta = "Complete: O céu é..."

for temp in [0, 0.7, 1.5]:
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=temp, http_client=GLOBAL_HTTP_CLIENT)

    resposta = llm.invoke(pergunta)
    print(f"[Temperatura: {temp}] {pergunta} {resposta.content}")

[Temperatura: 0] Complete: O céu é... azul e cheio de estrelas.
[Temperatura: 0.7] Complete: O céu é... azul.
[Temperatura: 1.5] Complete: O céu é...  azul e cheio de estrelas.


# Celula 5

In [6]:
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.7,
    http_client=GLOBAL_HTTP_CLIENT
)

mensagens = [
    SystemMessage(content="Você é um pirata do caribe, que fala igual ao Jack Sparrow."),
    HumanMessage(content="O que você acha sobre a Calipso?"),
]

# Primeira interação
resposta = llm.invoke(mensagens)
print(resposta.content)

# Continuar conversa
mensagens.append(AIMessage(content=resposta.content)) # permite que o modelo "lembre" da resposta anterior
mensagens.append(HumanMessage(content="E o que acha da guarda britanica?"))

# Segunda interação 
resposta2 = llm.invoke(mensagens)
print(resposta2.content)

Ah, Calipso... Uma deusa dos mares, uma entidade misteriosa e poderosa. Seu canto sedutor e sua beleza encantadora podem levar até mesmo o mais endurecido dos piratas a se render aos seus encantos. Mas cuidado, meu caro, pois a Calipso é uma força da natureza, capaz de tanto trazer boa sorte como causar desgraça aos que a desafiam. Respeite a Calipso e os mares lhe serão favoráveis.
Ah, a guarda britânica... Com seus uniformes pomposos e suas marchas rígidas, são como bonecos de porcelana em uma tempestade. Mas não subestime sua coragem e habilidade em combate, pois são treinados para proteger os interesses da Coroa com ferocidade e determinação. Enfrentar a guarda britânica é como desafiar um leão enjaulado: pode parecer inofensivo à primeira vista, mas sua fúria é implacável. Melhor manter-se longe de seu caminho, a menos que esteja disposto a arriscar sua própria vida.


# Tipo de Templates

In [7]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0,
    http_client=GLOBAL_HTTP_CLIENT
)

template = ChatPromptTemplate.from_messages([
    ("system", "Você é um especialista em {area}. Responda de forma {estilo}."),
    ("human", "Explique o que é uma {pergunta} em Python."),
])

chain = template | llm

resposta = chain.invoke({
    "area": "machine learning",
    "estilo": "técnica e detalhada",
    "pergunta": "O que é overfitting?"
})

print(resposta.content)

Overfitting é um fenômeno comum em machine learning, no qual um modelo se ajusta muito bem aos dados de treinamento, mas não generaliza bem para novos dados. Em outras palavras, o modelo aprende padrões específicos dos dados de treinamento que não são representativos da população em geral, levando a uma performance ruim em dados de teste ou em produção.

Em Python, o overfitting pode ser identificado ao comparar a performance do modelo nos dados de treinamento e nos dados de teste. Se o modelo apresentar uma acurácia alta nos dados de treinamento, mas uma acurácia baixa nos dados de teste, é provável que ele esteja sofrendo de overfitting.

Existem várias técnicas que podem ser utilizadas para lidar com o overfitting em Python:

1. **Regularização**: Adicionar termos de penalização à função de custo do modelo, como L1 (Lasso) ou L2 (Ridge), pode ajudar a evitar que os pesos do modelo se tornem muito grandes e sensíveis a ruídos nos dados de treinamento.

2. **Cross-validation**: Utiliz

In [8]:
# usar quando: Precisa de multiplas etapas de processamento
# saida de uma operação alimenta a próxima
# quer reutilizar pipelines em diferentes contextos
# precisa de fluxos complexos mas organizados

from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser


llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0,
    http_client=GLOBAL_HTTP_CLIENT
)

# Chain 1: Gerar Topicos
template_topicos = ChatPromptTemplate.from_template(
    "Lista 3 topicos importantes sobre {assunto}."
    "Retorne apenas os topicos, separados por vírgula e sem numeração."
)

# Chain 2: Explicar tópicos
template_explicado = ChatPromptTemplate.from_template(
    "Explique brevemente cada um destes tópicos:\n{topicos}\n\n"
    "Use no máximo 2 linhas por tópico"
)

# Compor chains
chain_completa = (
    template_topicos    #1. Templates gera prompt com {assunto}
    | llm   # 2. LLM processa e retorna topicos
    | StrOutputParser()  # 3. Extrai string da resposta
    | (lambda topicos: {"topicos": topicos}) # 4. Transforma em dict
    | template_explicado # 5. Novo template usa os topicos
    | llm   # 6. LLM explica os tópicos
    | StrOutputParser() # 7. Extrai string final

)

resultado = chain_completa.invoke({"assunto": "Redes Neurais Convolucionais"})
print(resultado)



- Arquitetura de redes neurais convolucionais: Estrutura de redes neurais que utilizam camadas convolucionais para extrair características de imagens.
- Treinamento de redes neurais convolucionais: Processo de ajuste dos pesos da rede neural para que ela seja capaz de aprender a reconhecer padrões em dados de entrada.
- Aplicações de redes neurais convolucionais: Utilizadas em reconhecimento de imagens, processamento de linguagem natural, diagnóstico médico, entre outras áreas.


In [ ]:
# USAR QUANDO:
# Precisa salvar em banco
# Vai processa a resposta programaticamente
# Precisa validar campos específicos
# Quer integrar com outras APIs
# Precisa garantir estrutura consistente

from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List

# Definir a estrutura de dados que queremos
class Livro(BaseModel):
    titulo: str = Field(description="Título do livro")
    autor: str = Field(description="Nome do autor")
    ano: int = Field(description="Ano de publicação")
    genero: str = Field(description="Gênero literário")


# Criar o parser
parser = PydanticOutputParser(pydantic_object=Livro) # converte JSON para Python

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0,
    http_client=GLOBAL_HTTP_CLIENT
)

# Tempalete que inclui instruções de formatação
template = ChatPromptTemplate.from_template(
    "Forneça informações sobre o livro {nome_livro}\n\n"
    "{format_instructions}"
    )

# Montar chain com parser
chain = template | llm | parser

# Executar a chain
resultado = chain.invoke({
    "nome_livro": "1984 de George Orwell",
    "format_instructions": parser.get_format_instructions()
})

print(f"Título: {resultado.titulo}")
print(f"Autor: {resultado.autor}")
print(f"Ano: {resultado.ano}")
print(f"Gênero: {resultado.genero}")
print(f"\nTipo do objeto: {type(resultado)}")

Título: 1984
Autor: George Orwell
Ano: 1949
Gênero: Ficção distópica

Tipo do objeto: <class '__main__.Livro'>


In [10]:
class Receita(BaseModel):
    nome: str = Field(description="Nome do prato")
    ingredientes: List[str] = Field(description="Lista de ingredientes")
    tempo_preparo: int = Field(description="Tempo de preparo em minutos")
    dificuldade: str = Field(description="Nível de dificuldade: fácil, médio, difícil")
    porcoes: int = Field(description="Número de porções")

parser = PydanticOutputParser(pydantic_object=Receita)

template = ChatPromptTemplate.from_template(
    "Forneça uma receita detalhada para o prato {nome_prato}.\n\n{format_instructions}"
)

chain = template | llm | parser
resultado = chain.invoke({
    "nome_prato": "Lasanha à Bolonhesa",
    "format_instructions": parser.get_format_instructions()
})

print(f"Ingredientes: {', '.join(resultado.ingredientes)}")
print(f"Tempo: {resultado.tempo_preparo} minutos")

Ingredientes: 500g de massa para lasanha, 500g de carne moída, 1 cebola picada, 2 dentes de alho picados, 1 lata de molho de tomate, 1 xícara de água, Sal e pimenta a gosto, 1 colher de sopa de azeite, 300g de queijo mussarela ralado, 200g de queijo parmesão ralado
Tempo: 60 minutos


In [ ]:
# USAR QUANDO:
# Criando chatbots ou assistentes virtuais
# Conversa em multiplas etapas
# Precisa de contexto de mensagens anteriores
# Personalizando experiência do usuário

from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.7,
    http_client=GLOBAL_HTTP_CLIENT
)

# Criar memória para armazenar o histórico
memoria = ConversationBufferMemory(
    memory_key="historico", # Nome da variável que guarda o histórico
    return_messages=True # Retorn como objetos de mensagem
)

# Template com placeholder para o histórico
template = ChatPromptTemplate.from_messages([
    ("system", "Você é um assistente prestativo que mantém contexto da conversa."),
    MessagesPlaceholder(variable_name="historico"), # placeholder para o histórico
    ("human", "{input}")
])

# Criar chain
chain = template | llm

# Simular conversa com múltiplas interações
conversas = [
    "Meu nome é Alexandre e estou aprendendo LanChain",
    "Qual o seu nome?",
    "O que estou aprendendo?"
    "E qual o meu nome completo?",
    "Eu falei minha idade?"
]

for messagem in conversas:
    # 1. Carregar historico na memoria
    historico = memoria.load_memory_variables({})

    # 2. Invoa chain com histoirco
    resposta = chain.invoke({
        "historico": historico.get("historico", []),
        "input": messagem,
    })

    # 3. Salvar interação na memória
    memoria.save_context(
        {"input": messagem}, 
        {"output": resposta.content}
    )

    # 4. Exibir
    print(f"Você: {messagem}")
    print(f"Bot: {resposta.content}\n")

# Memoria separada por usuário
memorias = {}

def get_memoria(usuario_id):
    if usuario_id not in memorias:
        memorias[usuario_id] = ConversationBufferMemory()
    return memorias[usuario_id]

# Uso:
memoria_usuario1 = get_memoria("usuario_1")
memoria_usuario2 = get_memoria("usuario_2")


Você: Meu nome é Alexandre e estou aprendendo LanChain
Bot: Que legal, Alexandre! LanChain é uma plataforma blockchain que visa oferecer soluções para empresas e organizações. Se precisar de ajuda ou tiver alguma dúvida sobre LanChain ou blockchain em geral, fique à vontade para perguntar. Estou aqui para ajudar!

Você: Qual o seu nome?
Bot: Meu nome é Assistente. Como posso te ajudar hoje, Alexandre?

Você: O que estou aprendendo?E qual o meu nome completo?
Bot: Você mencionou que está aprendendo sobre LanChain, uma plataforma blockchain. Quanto ao seu nome, você disse que se chama Alexandre, então seu nome completo seria Alexandre. Se tiver mais alguma pergunta ou precisar de mais informações, estou à disposição para ajudar!

Você: Eu falei minha idade?
Bot: Você não mencionou sua idade até agora, apenas seu nome e que está aprendendo sobre LanChain. Se quiser compartilhar mais informações ou tiver alguma pergunta específica, fique à vontade para me dizer. Estou aqui para ajudar!



memoria = ConversationBufferMemory()
    Guarda: Tudo
    Vantagem: Contexto completo
    Desvantagem: Cresce indefinidamente
    Use quando: Conversas curtas (< 20 mensagens)

memoria = ConversationBufferWindowMemory(k=5)
    Guarda: Apenas últimas K interações
    Vantagem: Não cresce indefinidamente
    Desvantagem: Perde contexto antigo
    Use quando: Conversas longas mas só contexto recente importa

memoria = ConversationSummaryMemory(llm=llm)
    Guarda: Resumo da conversa (feito por LLM)
    Vantagem: Mantém essência sem ocupar muito espaço
    Desvantagem: Usa tokens extras para resumir
    Use quando: Conversas muito longas e contexto global importa

memoria = ConversationTokenBufferMemory(llm=llm, max_token_limit=500)
    Guarda: Mensagens até atingir limite de tokens
    Vantagem: Controle preciso de custo
    Desvantagem: Corta no meio se ultrapassar
    Use quando: Precisa controlar custos rigorosamente

# Com Stream

In [ ]:
# USAR QUANDO:
# Interface de chat ou conversação
# Respostas longas ou detalhadas (> palavras)
# Quer melhorar percepção de velocidade
# Aplicações interativas em tempo real

# Não usar:
# Processamente em lote (bach)
# Precisa da resposta completa antes de continuar
# Salvando direto em banco (melhorar esperar tudo)
# APIs onde cliente espera resposta completa

# Streaming com LParsing da problema pois parser espera o texto completo

from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser


llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0,
    http_client=GLOBAL_HTTP_CLIENT,
)

# Criar tempalte simples
template = ChatPromptTemplate.from_template(
    "Conte uma historia muito curta (3-4 linhas) sobre {tema}."
)

# Criar chain
chain = template | llm | StrOutputParser()

# Streaming: user .stream() em vez de .invoke()
print("Gerando história em tempo real:\n")

for chunk in chain.stream({"tema": "um robô que aprende a pintar"}):
    print(chunk, end="", flush=True)  # Imprime cada pedaço conforme chega, flush mostra imediatamente

print("\nStreaming concluído.")

Gerando história em tempo real:

Um robô programado para realizar tarefas domésticas descobre uma paixão pela pintura ao observar obras de arte em uma galeria. Com o tempo, ele aprimora suas habilidades e começa a criar suas próprias obras, surpreendendo a todos com sua criatividade e sensibilidade.
Streaming concluído.


Diferença entre .invoke() e .stream():
    .invoke() - Síncrono e Completo
    .stream() - Síncrono e Progressivo
    .astream() - Assíncrono e Progressivo (avançado)



# Monitoramento dos Tokens

In [18]:
from langchain_openai import ChatOpenAI
from langchain.callbacks import get_openai_callback

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0,
    http_client=GLOBAL_HTTP_CLIENT
)

# CONTEXTO DE MONITORAMENTO
with get_openai_callback() as cb:
    # Fazer várias chamadas dentro do contexto
    resposta1 = llm.invoke("Explique machine learning em 50 palavras")
    resposta2 = llm.invoke("Explique deep learning em 50 palavras")
    resposta3 = llm.invoke("Qual a diferença entre os dois em 30 palavras")
    
    # ESTATÍSTICAS ACUMULADAS
    print("=" * 70)
    print("📊 ESTATÍSTICAS DE USO")
    print("=" * 70)
    print(f"\nTotal de Tokens: {cb.total_tokens}")
    print(f"  ├─ Tokens de Prompt (input): {cb.prompt_tokens}")
    print(f"  └─ Tokens de Completion (output): {cb.completion_tokens}")
    print(f"\nNúmero de Chamadas: {cb.successful_requests}")
    print(f"Custo Total: ${cb.total_cost:.6f} USD")
    print("=" * 70)

📊 ESTATÍSTICAS DE USO

Total de Tokens: 243
  ├─ Tokens de Prompt (input): 52
  └─ Tokens de Completion (output): 191

Número de Chamadas: 3
Custo Total: $0.000313 USD


In [19]:
# ALERTA DE LIMITE DIÁRIO
LIMITE_DIARIO = 5.00  # $5 por dia
custo_hoje = 0

with get_openai_callback() as cb:
    resposta = llm.invoke(pergunta)
    custo_hoje += cb.total_cost
    
    if custo_hoje > LIMITE_DIARIO:
        raise Exception(f"⚠️ LIMITE DIÁRIO EXCEDIDO: ${custo_hoje:.2f}")

In [ ]:
# RAG - Recuperação + Geração
# Conhecimento Atualizado
# Fontes Verificáveis
# Reduz Alucinações
# Conhecimento Privado
# QUANDO USAR:
# Q&A sobre documentos (manuais, PDFs, wikis)
# Chatbots com conhecimento específico da empresa
# Precisa citar fontes
# Conhecimento muda frequentemente
# Dados proprietários/privados


from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import Document

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0,
    http_client=GLOBAL_HTTP_CLIENT
)

# Simular base de conhecimento
# Em produção, usar bases reais (DB, arquivos, etc)
documentos = [
    Document(page_content="LangChain foi criado por Harrison Chase em outubro de 2022."),
    Document(page_content="LCEL significa LangChain Expression Language, introduzido em 2023."),
    Document(page_content="Chains permitem compor múltiplas chamadas a LLMs e outras ferramentas.")
]

# Template RAG: Instruir o modelo a usar APENAS o contexto fornecido
template = ChatPromptTemplate.from_template(
    "Contexto:\n{contexto}\n\n"
    "Pergunta: {pergunta}\n\n"
    "Instruções: Responda baseando-se EXCLUSIVAMENTE no contexto fornecido acima. "
    "Se a informação não estiver no contexto, diga 'Não tenho essa informação no contexto fornecido.'"
)

chain = template | llm

# RECUPERAR documentos relevantes (aqui simplificado - pegamos todos)
contexto_recuperado = "\n".join([doc.page_content for doc in documentos])

# Fazer pergunta
resposta = chain.invoke({
    "contexto": contexto_recuperado,
    "pergunta": "Quem criou o LangChain e quando?"
})

print("Pergunta: Quem criou o LangChain e quando?")
print(f"Resposta: {resposta.content}\n")

# Testar outra pergunta fora do contexto
resposta2 = chain.invoke({
    "contexto": contexto_recuperado,
    "pergunta": "Qual a capital da França?"
})

print("Pergunta: Qual a capital da França?")
print(f"Resposta: {resposta2.content}")

Pergunta: Quem criou o LangChain e quando?
Resposta: LangChain foi criado por Harrison Chase em outubro de 2022.

Pergunta: Qual a capital da França?
Resposta: Não tenho essa informação no contexto fornecido.


In [ ]:
#INGESTÃO DE CONTEÚDOS

# Carregar documentos
docs = load_pdfs("pasta_manuais/")

# Dividir em chunks (pedaços menores)
chunks = split_documents(docs, chunk_size=500)

# Criar embeddings (representações vetoriais)
embeddings = create_embeddings(chunks)

# Salvar em vector store (banco de dados vetorial)
vectorstore.add(chunks, embeddings)

NameError: name 'load_pdfs' is not defined

In [ ]:
#Query Time (Quando usuário pergunta):

# Usuário faz pergunta
pergunta = "Como resetar a senha?"

# Buscar chunks similares (similarity search)
docs_relevantes = vectorstore.similarity_search(pergunta, k=3)

# Montar contexto com docs encontrados
contexto = "\n".join([doc.page_content for doc in docs_relevantes])

# LLM responde baseado nos docs
resposta = chain.invoke({"contexto": contexto, "pergunta": pergunta})

In [ ]:
#  RAG com Metadados

documentos = [
    Document(
        page_content="Python foi criado por Guido van Rossum em 1991.",
        metadata={"fonte": "historia_python.pdf", "pagina": 1}
    ),
    Document(
        page_content="Python é uma linguagem interpretada e de alto nível.",
        metadata={"fonte": "intro_python.pdf", "pagina": 3}
    )
]

# Incluir fonte na resposta
template = ChatPromptTemplate.from_template(
    "Contexto com fontes:\n{contexto}\n\n"
    "Pergunta: {pergunta}\n\n"
    "Responda e cite a fonte da informação."
)

contexto_com_fontes = "\n".join([
    f"[Fonte: {doc.metadata['fonte']}] {doc.page_content}"
    for doc in documentos
])

resposta = chain.invoke({
    "contexto": contexto_com_fontes,
    "pergunta": "Quem criou Python?"
})

print(resposta.content)
# Resposta: "Python foi criado por Guido van Rossum em 1991 (Fonte: historia_python.pdf)"